In [103]:
import os
import sys

In [104]:
%pwd

'E:\\'

In [3]:
os.chdir("../")

In [4]:
%pwd

'E:\\6_learning\\Kidney-Disease-Classification-DLproject'

In [127]:
# entity >> return type of data ingestion fuction

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [128]:
# Go 2 levels up from research/ to project root
ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

In [41]:
!pip install -e .

Obtaining file:///E:/6_learning/Kidney-Disease-Classification-DLproject
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for src


  DEPRECATION: Legacy editable install of src==0.0.0 from file:///E:/6_learning/Kidney-Disease-Classification-DLproject (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [129]:
from src.constants import *
from src.utils.common import create_directories, read_yml

In [130]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yml(config_filepath)
        self.params = read_yml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [131]:
import os
import zipfile
import gdown
from src import logging
from src.utils.common import get_size

In [133]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[5]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logging.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [134]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2025-08-13 10:51:56,809 ] 31 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2025-08-13 10:51:56,813 ] 31 root - INFO - yaml file: params.yaml loaded successfully
[ 2025-08-13 10:51:56,816 ] 50 root - INFO - created directory at: artifacts
[ 2025-08-13 10:51:56,822 ] 50 root - INFO - created directory at: artifacts/data_ingestion
[ 2025-08-13 10:51:56,824 ] 15 root - INFO - Downloading data from https://drive.google.com/file/d/1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl/view?usp=sharing into file artifacts/data_ingestion/data.zip
Downloading...
From (original): https://drive.google.com/uc?export=download&id=1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl
From (redirected): https://drive.google.com/uc?export=download&id=1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl&confirm=t&uuid=f69e791d-ed90-4ebd-80a3-44290c26386f
To: E:\6_learning\Kidney-Disease-Classification-DLproject\artifacts\data_ingestion\data.zip
100%|█████████████████████████████████████████████████████████████████████████████| 38.9M/38.9M [

In [112]:
import os
os.chdir(r"E:\6_learning\Kidney-Disease-Classification-DLproject")
print(os.getcwd())  # should now show the project root

E:\6_learning\Kidney-Disease-Classification-DLproject
